In [1]:
from pyscipopt import Model, quicksum, multidict

In [2]:
def FLP(I, J, d, M, f, C):
    model=Model("FLP")
    x,y={}, {}
    for j in J:
        y[j]=model.addVar(vtype="B")
        for i in I:
            x[i,j]=model.addVar(vtype="C")
            
    for i in I :
        model.addCons(quicksum(x[i,j]for j in J)==d[i])
        
    for j in M :
        model.addCons(quicksum(x[i,j] for i in I)<= M[j]*y[j])
    
    for (i,j) in x :
        model.addCons( x [i,j] <= d[i]*y[j])
        
    model.setObjective(quicksum(f[j]*y[j]for j in J) + quicksum(C[i,j]*x[i,j] for i in I for j in J), "minimize")
    model.data = x,y
    return model
        
   


In [4]:
 #Demand by each customer 
 I, d = multidict({1:3, 2:5, 3:1, 4:8, 5:4})
 
 # fixed cost and maximum capacity of each facillity 
 J, M, f = multidict({1:[12,250000], 2:[11,200000], 3:[10,280000], 4:[14,220000]})
 
 #Transportation cost between facilities and customers 
 C = {(1,1):300,  (1,2):700,  (1,3):200, (1,4):300,
      (2,1):200,  (2,2):300,  (2,3):400, (2,4):300,
      (3,1):400,  (3,2):500,  (3,3):100, (3,4):400,
      (4,1):200,  (4,2):400,  (4,3):600, (4,4):600,
      (5,1):400, (5,2):500,  (5,3):300, (5,4):400,}

In [5]:
model=FLP(I, J, d, M, f, C)
model.optimize()
x,y = model.data 

edges = [ (i,j) for (i,j) in x if model.getVal(x[i,j]) > 0.0001]
facilities= [ j for j in y if model.getVal(y[j])> 0.0001]
quantities = { (i,j) : model.getVal(x[i,j]) for (i,j) in x }

In [6]:
print("Optimal Cost", model.getObjVal())
print("Facilites to open", facilities)

for (i, j), quantity in quantities.items():
    if quantity > 0.0001:
        print(f"Quantity from Facility {j} to Customer {i}: {quantity:.2f} Ton")

Optimal Cost 427600.00000000006
Facilites to open [2, 4]
Quantity from Facility 2 to Customer 4: 8.00 Ton
Quantity from Facility 4 to Customer 1: 3.00 Ton
Quantity from Facility 4 to Customer 2: 5.00 Ton
Quantity from Facility 4 to Customer 3: 1.00 Ton
Quantity from Facility 4 to Customer 5: 4.00 Ton
